In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np

from trustee import ClassificationTrustee

In [ ]:
attacker_ips = {
    '169.231.210.93',
    '169.231.28.232',
    '169.231.123.195',
    '169.231.172.165',
    '169.231.11.193',
    '169.231.8.190',
    '169.231.10.199',
}

In [ ]:
campus_dataset = pd.read_csv('campus_dataset_1.csv')
campus_dataset['Class'] = 0
campus_dataset.loc[campus_dataset['Src IP'].isin(attacker_ips), 'Class'] = 1

print(len(campus_dataset))
ttl_data = pd.read_csv('campus_ttl_1.csv')
campus_dataset = campus_dataset.merge(ttl_data, on="Flow ID", how='left')
len(campus_dataset)

In [ ]:
campus_dataset = campus_dataset.drop([
    'Flow ID',
    'Src IP',
    'Dst IP',
    'Timestamp', 
    'Protocol',    # always tcp
    'Label',       # empty
], axis=1)

In [ ]:
azure_dataset = pd.read_csv('azure_dataset_1.csv')
azure_dataset['Class'] = 0
azure_dataset.loc[azure_dataset['Src IP'].isin(attacker_ips), 'Class'] = 1
ttl_df_azure = pd.read_csv('azure_ttl_1.csv')
azure_dataset = azure_dataset.merge(ttl_df_azure, on="Flow ID", how='left')
azure_dataset = azure_dataset.replace([np.inf, -np.inf], np.nan)
azure_dataset = azure_dataset.dropna(axis=0)

In [ ]:
attacker_ips = {
    '157.245.108.149',
    '34.214.149.122',
}

multicloud_dataset = pd.read_csv('multicloud_dataset_1.csv')
multicloud_dataset['Class'] = 0
multicloud_dataset.loc[multicloud_dataset['Src IP'].isin(attacker_ips), 'Class'] = 1
ttl_df_multicloud = pd.read_csv('multicloud_ttl_1.csv')
multicloud_dataset = multicloud_dataset.merge(ttl_df_multicloud, on="Flow ID", how='left')
multicloud_dataset = multicloud_dataset.replace([np.inf, -np.inf], np.nan)
multicloud_dataset = multicloud_dataset.dropna(axis=0)

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

Train on campus dataset, check on Azure and multicloud

In [ ]:
target_variable = 'Class'
features = list(set(campus_dataset.columns) - {target_variable})
x_train = campus_dataset[features]
y_train = campus_dataset[target_variable]
x_test = azure_dataset[features]
y_test = azure_dataset[target_variable]
x_test_2 = multicloud_dataset[features]
y_test_2 = multicloud_dataset[target_variable]

In [ ]:
from sklearn.preprocessing import StandardScaler

x_train = pd.DataFrame(StandardScaler().fit_transform(x_train), columns = x_train.columns)
x_test = pd.DataFrame(StandardScaler().fit_transform(x_test), columns = x_test.columns)
x_test_2 = pd.DataFrame(StandardScaler().fit_transform(x_test_2), columns = x_test_2.columns)

In [ ]:
classifiers = [
    MLPClassifier(),
    GradientBoostingClassifier(),
    RandomForestClassifier(),
]

In [ ]:
for clf in classifiers:
    print(clf)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_train)
    print("campus dataset training accuracy: ")
    print(metrics.classification_report(y_train, y_pred))

    print("Azure dataset test accuracy: ")
    y_pred = clf.predict(x_test)
    print(metrics.classification_report(y_test, y_pred))
    print(metrics.confusion_matrix(y_test, y_pred))
    
    print("Multicloud dataset test accuracy: ")
    y_pred = clf.predict(x_test_2)
    print(metrics.classification_report(y_test_2, y_pred))
    print(metrics.confusion_matrix(y_test_2, y_pred))
    print('#' * 10 + '\n')

Let's explore the reasons of this performance

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)

y_pred = clf.predict(x_train)
print("campus dataset training accuracy: ")
print(metrics.classification_report(y_train, y_pred))

y_pred = clf.predict(x_test)
print("Azure dataset test accuracy: ")
print(metrics.classification_report(y_test, y_pred))
print(metrics.confusion_matrix(y_test, y_pred))

y_pred = clf.predict(x_test_2)
print("Multicloud dataset test accuracy: ")
print(metrics.classification_report(y_test_2, y_pred))
print(metrics.confusion_matrix(y_test_2, y_pred))

fig = plt.figure(figsize=(25,20))
_ = plot_tree(clf, feature_names=x_train.columns, class_names=['benign', 'attack'], filled=True, max_depth=2)

The answer is a shortcut in the dataset - usage of TTL as a feature.  
Random forests choose features randomly and therefore often remove TTL from the list and doesn't overfit to it.  

Anyway, TTL is a plain shortcut, let's remove it.

## Out of curiosity - dropping or noising the features
Use information from the future - we know that TTL and Init Win Bytes are or would be shortcuts, so let's add some noise there or drop them and check performance.

In [ ]:
target_variable = 'Class'
features = list(set(campus_dataset.columns) - {target_variable})
x_train = campus_dataset[features].copy()
y_train = campus_dataset[target_variable]
x_test = azure_dataset[features]
y_test = azure_dataset[target_variable]
x_test_2 = multicloud_dataset[features]
y_test_2 = multicloud_dataset[target_variable]

In [ ]:
x_train.loc[:, 'TTL'] += np.random.randint(-1, 1, [len(x_train)])
x_train.loc[:, 'Bwd Init Win Bytes'] += np.random.randint(-5, 5, [len(x_train)])

In [ ]:
x_train = pd.DataFrame(StandardScaler().fit_transform(x_train), columns = x_train.columns)
x_test = pd.DataFrame(StandardScaler().fit_transform(x_test), columns = x_test.columns)
x_test_2 = pd.DataFrame(StandardScaler().fit_transform(x_test_2), columns = x_test_2.columns)

In [ ]:
classifiers = [
    MLPClassifier(),
    GradientBoostingClassifier(),
    RandomForestClassifier(),
]
for clf in classifiers:
    print(clf)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_train)
    print("campus dataset training accuracy: ")
    print(metrics.classification_report(y_train, y_pred))

    print("Azure dataset test accuracy: ")
    y_pred = clf.predict(x_test)
    print(metrics.classification_report(y_test, y_pred))
    print(metrics.confusion_matrix(y_test, y_pred))
    
    print("Multicloud dataset test accuracy: ")
    y_pred = clf.predict(x_test_2)
    print(metrics.classification_report(y_test_2, y_pred))
    print(metrics.confusion_matrix(y_test_2, y_pred))
    print('#' * 10 + '\n')

### Or let's just drop them

In [ ]:
x_train = x_train.drop(['TTL', 'Bwd Init Win Bytes'], axis=1)
x_test = x_test.drop(['TTL', 'Bwd Init Win Bytes'], axis=1)
x_test_2 = x_test_2.drop(['TTL', 'Bwd Init Win Bytes'], axis=1)

In [ ]:
classifiers = [
    MLPClassifier(),
    GradientBoostingClassifier(),
    RandomForestClassifier(max_depth=4),
]
for clf in classifiers:
    print(clf)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_train)
    print("campus dataset training accuracy: ")
    print(metrics.classification_report(y_train, y_pred))

    print("Azure dataset test accuracy: ")
    y_pred = clf.predict(x_test)
    print(metrics.classification_report(y_test, y_pred))
    print(metrics.confusion_matrix(y_test, y_pred))
    
    print("Multicloud dataset test accuracy: ")
    y_pred = clf.predict(x_test_2)
    print(metrics.classification_report(y_test_2, y_pred))
    print(metrics.confusion_matrix(y_test_2, y_pred))
    print('#' * 10 + '\n')

In [ ]:
trustee = ClassificationTrustee(expert=classifiers[0])
trustee.fit(x_train, y_train, num_iter=10, num_stability_iter=3, samples_size=0.8)

_, dt, _, score = trustee.explain()
print(f"Training score of pruned DT: {score}")
dt_y_pred = dt.predict(x_test)

print("Model explanation global fidelity report:")
print(metrics.classification_report(classifiers[0].predict(x_test), dt_y_pred))
print("Model explanation score report:")
print(metrics.classification_report(y_test, dt_y_pred))

fig = plt.figure(figsize=(25,20))
plot_tree(dt, feature_names=x_train.columns, class_names=['benign', 'attack'], filled=True, max_depth=3)